In [1]:
import pandas as pd
import genanki
from pathlib import Path

waza = pd.read_csv("../data/datasets/waza.csv")
waza = waza.sort_values(
    ["group_id", "group_index", "category_id", "category_index"]
).reset_index(drop=True)
waza

,name,translated_name,category,category_id,category_index,group,group_id,group_index,filename
0,de-ashi-harai,Forward Foot Sweep,ashi-waza,3,0,dai-ikkyo,1,1,judo_3-00_ashi-waza_de-ashi-harai.gif
1,hiza-guruma,Knee Wheel,ashi-waza,3,1,dai-ikkyo,1,2,judo_3-01_ashi-waza_hiza-guruma.gif
2,sasae-tsurikomi-ashi,Propping-Lifting Pulling Ankle Throw,ashi-waza,3,2,dai-ikkyo,1,3,judo_3-02_ashi-waza_sasae-tsurikomi-ashi.gif
3,uki-goshi,Floating Hip,koshi-waza,2,0,dai-ikkyo,1,4,judo_2-00_koshi-waza_uki-goshi.gif
4,o-soto-gari,Major Outer Reaping,ashi-waza,3,3,dai-ikkyo,1,5,judo_3-03_ashi-waza_o-soto-gari.gif
...,...,...,...,...,...,...,...,...,...
105,ude-hishigi-hara-gatame,Arm Lock with Stomach,kansetsu-waza,8,5,uncategorized,8,0,judo_8-05_kansetsu-waza_ude-hishigi-hara-gatam...
106,ude-hishigi-ashi-gatame,Arm Lock with Leg,kansetsu-waza,8,6,uncategorized,8,0,judo_8-06_kansetsu-waza_ude-hishigi-ashi-gatam...
107,ude-hishigi-te-gatame,Arm Lock with Hand,kansetsu-waza,8,7,uncategorized,8,0,judo_8-07_kansetsu-waza_ude-hishigi-te-gatame.gif
108,ude-hishigi-sankaku-gatame,Triangle Arm Lock,kansetsu-waza,8,8,uncategorized,8,0,judo_8-08_kansetsu-waza_ude-hishigi-sankaku-ga...


Lets generate cards in this order

- by group (gokyo no waza)
    - technique recognition
    - technique translation

- https://languagelearning.stackexchange.com/questions/5610/which-direction-to-use-for-memorizing-new-words

In [2]:
import random

random.seed(33)
for i in range(2):
    print(random.randrange(1 << 30, 1 << 31))

1432916526
1574573024


In [3]:
# make the image take up the width of the card
center = (
    "<style>div { text-align: center; } img { width: 100%; max-width: 400px; }</style>"
)
model = genanki.Model(
    1432916526,
    "ImageNameRecall",
    fields=[{"name": "Image"}, {"name": "Name"}, {"name": "TranslatedName"}],
    templates=[
        {
            "name": "Card 1",
            "qfmt": center + "<div>{{Image}}</div>",
            "afmt": center
            + '{{FrontSide}}<hr id="answer"><div><h1>{{Name}}</h1></div>',
        },
        {
            "name": "Card 2",
            "qfmt": center + "<div><h1>{{TranslatedName}}</h1></div>",
            "afmt": center
            + '{{FrontSide}}<hr id="answer"><div><h1>{{Name}}</h1></div>',
        },
    ],
)

deck = genanki.Deck(
    1574573024,
    "Judo Techniques",
    (f"An Anki deck for recognition of Judo techniques."),
)

masked_path = Path("../data/clips/waza-masked-gif")
for row in waza.itertuples():
    filename = (
        f"judo_{row.category_id}-{row.category_index:02d}_{row.category}_{row.name}.gif"
    )
    assert masked_path.exists(), filename
    deck.add_note(
        genanki.Note(
            model=model,
            fields=[f'<img src="{filename}">', row.name, row.translated_name],
        )
    )

package = genanki.Package(deck)
package.media_files = sorted(masked_path.glob("*.gif"))
package.write_to_file("../data/judo-techniques-20230820.apkg")